In [1]:
import sys
import os
import time
import operator
import pandas as pd

class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'ID: {}, count: {}'.format(self.ID,self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self):
        super().__init__()
        self.distinct = hyperloglog.HyperLogLog(0.01)
    def __str__(self):
        return 'total count: {}, distinct element: {}'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return 'ID: {}, count: {}'.format(self.ID,self.count)
    def __repr__(self):
        return "'{}', count: {}".format(self.ID,self.count)

def find(e,element_list):
    try:
        index = [ele.ID for ele in element_list].index(e.ID)
    except:
        index=-99
    return index  
    
    
start=time.time()

filename='kosarak.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)

size=820
Top=[]
#item_count=100000

with open(src_data,'r') as file:
    while True:
        element=file.readline().strip('\n')
        if not element:
            print('EOF')
            break
        else:
            item=Tail(element,1)
            #item_count-=1
            # print("read {}th element: {}".format(item_count,element))
            if len(Top)==0:
                Top.append(item)
            else:
                index=find(item,Top)
                if index<0:
                    if len(Top)<size:
                        Top.append(item)
                    else:
                        # replace last element with count 
                        Top[-1].ID=item.ID
                        Top[-1].count+=1
                else:
                    Top[index].count+=1
            Top.sort(key=operator.attrgetter('count'),reverse=True)

end=time.time()
print(Top[:20],len(Top))
print("Total memory {0} bytes :Top-{1} with size {0} bytes.".format(sys.getsizeof(Top),size))
print("Execution time:{} seconds.".format(str(end-start)))


#　conver Top into df    
templi=[]
for i in Top:
    templi.append([i.ID,i.count])

df=pd.DataFrame(templi,columns=['ID', 'Count'])
df.to_csv('..\\result\\kosarak\\SS_class_kosarak'+str(size)+'.csv',index=False)
df.head(20)

#====================result compare=============================
path='..\\result\\kosarak'
groundtruth='kosarak_ground_truth.csv'
final='SS_class_kosarak'+str(size)+'.csv'

grtruth=pd.read_csv(os.path.join(path,groundtruth))
My_result1=pd.read_csv(os.path.join(path,final))

gli = grtruth.values.tolist()
li1= My_result1.values.tolist()

tp=0
fp=0
find=0
err=[]
error=0
for item in li1:
    for element in gli[:size]:
        if item[0]==element[0]:
            # print("{},{} vs. {},{}".format(item[0],item[1],element[0],element[1]))
            find+=1
            if item[1]==element[1]:
                tp+=1
            else:
                fp+=1
                error+=abs(item[1]-element[1])/item[1]
print("Find:{},TP:{},FP:{}".format(find,tp,fp))
#print("ARE:{}".format(error/606770))


EOF
['3', count: 36133, '6', count: 30225, '6 3', count: 21139, '11 6', count: 12769, '1', count: 9766, '11 6 3', count: 9083, '1 3', count: 6002, '11', count: 5374, '1 6', count: 5046, '4', count: 4290, '11 3', count: 3756, '55', count: 3561, '1 6 3', count: 3510, '4 3', count: 3000, '55 3', count: 2377, '11 1 6', count: 2375, '4 6', count: 2341, '4 6 3', count: 1758, '11 1 6 3', count: 1754, '6 55', count: 1750] 820
Total memory 7056 bytes :Top-820 with size 7056 bytes.
Execution time:183.33592295646667 seconds.
Find:112,TP:19,FP:93
